In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import helper

In [3]:
train_data = helper.load_data_from_file("../data/graph_data.json") # dictionary of SMILES and values are graphs

In [5]:
mol = list(train_data.keys())[14]
mol

'B(C)(C)C'

In [6]:
graph = train_data[mol]

In [ ]:
atom = list(nx.get_node_attributes(graph, "atom_type").values())

dict_values(['B', 'C', 'C', 'C'])

In [10]:
nx.get_node_attributes(graph, "orbitals")

{0: ['1s'], 1: [-1], 2: [-1], 3: [-1]}

In [12]:
nx.get_node_attributes(graph, "binding_energies")

{0: [196.4], 1: [-1], 2: [-1], 3: [-1]}